In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from keras.models import Model

In [2]:
pwd

'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10'

In [3]:
#data path
trainpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\train'#
testpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_10\\fruitclassification\\test'

In [4]:
new_size=224
train_images=[]
train_labels=[]
for i in os.listdir(trainpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(trainpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      train_images.append(list(resize_image))
      train_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 20
Entering to the folder name: gala
Number of images in the folder is 21
Entering to the folder name: grannysmith
Number of images in the folder is 20
Entering to the folder name: lime
Number of images in the folder is 20
Entering to the folder name: pinklady
Number of images in the folder is 20


In [5]:
new_size=224
test_images=[]
test_labels=[]
for i in os.listdir(testpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(testpath+'/' + i + '/*.png'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_fruit={'banana':0,'gala':1,'grannysmith':2,'lime':3,'pinklady':4}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      test_images.append(list(resize_image))
      test_labels.append(class_fruit[i])

Entering to the folder name: banana
Number of images in the folder is 10
Entering to the folder name: gala
Number of images in the folder is 10
Entering to the folder name: grannysmith
Number of images in the folder is 10
Entering to the folder name: lime
Number of images in the folder is 10
Entering to the folder name: pinklady
Number of images in the folder is 10


In [6]:
def list_to_array_train(train_images,train_labels):
  return np.array(train_images),np.array(train_labels)

train,train_labels=list_to_array_train(train_images,train_labels)


def list_to_array_test(test_images,test_labels):
  return np.array(test_images),np.array(test_labels)


test_images,test_labels=list_to_array_test(test_images,test_labels)

In [7]:
print(train_images.shape)
print("*"*20)
print(train_labels.shape)
print("*"*20)
print(test_images.shape)
print(test_labels.shape)

(101, 224, 224, 3)
********************
(101,)
********************
(50, 224, 224, 3)
(50,)


In [8]:
def keras_to_categorical(y_train,y_test):
  return to_categorical(y_train),to_categorical(y_test)

train_labels,test_labels=keras_to_categorical(test_images,test_labels)

In [9]:
def model_vgg19():
  VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
    
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  d1=Dense(units=128,kernel_initializer="glorot_uniform", activation='relu')(feature)
  d2=Dense(units=64,kernel_initializer="glorot_uniform", activation='sigmoid')(d1)
  d3=Dense(units=32,kernel_initializer="glorot_uniform", activation='sigmoid')(d2)
  d4=Dense(units=5,kernel_initializer="glorot_uniform", activation='softmax')(d3)
  output = Model(inputs =VGG_model.input, outputs =d4)
  #output = Model(inputs =VGG_model.input, outputs =feature)
  
  return output

model19=model_vgg19()

In [10]:
model19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
accuracies = []

# Define the number of runs
num_runs = 3

for run in range(num_runs):
    # Fit the model
    history = model19.fit(train_images,train_labels ,validation_split=0.2, epochs=10, batch_size=5, verbose=2, validation_data=(test_images, test_labels))
    
    # Evaluate the model on the test set
    _, accuracy = model19.evaluate(test_images, test_labels, verbose=2)
    
    # Append the accuracy to the list
    accuracies.append(accuracy)
    
    # Print the accuracy for each run
    print(f"Accuracy for run {run + 1}: {accuracy}")

# Calculate the mean accuracy
mean_accuracy = np.mean(accuracies)

# Print the mean accuracy
print(f"Mean accuracy across {num_runs} runs: {mean_accuracy}")

ValueError: Data cardinality is ambiguous:
  x sizes: 101
  y sizes: 50
Make sure all arrays contain the same number of samples.